In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash import Input, Output, State  
import base64
JupyterDash.infer_jupyter_proxy_config()  

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import CRUD_Python_Module
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "Candy340"  

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True, errors='ignore')  

# EXACT BREEDS FROM DASHBOARD SPECIFICATIONS DOCUMENT (PDF Table)
RESCUE_BREEDS = {
    'Water Rescue': {
        'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'sex': 'Intact Female',
        'age_min': 26, 'age_max': 156  # 26-156 weeks
    },
    'Mountain or Wilderness': {
        'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_min': 26, 'age_max': 156
    },
    'Disaster Rescue or Individual Tracking': {
        'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_min': 20, 'age_max': 300  # 20-300 weeks
    }
}

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)  # FIXED: JupyterDash for notebook display

#FIX ME Add in Grazioso Salvare's logo
image_filename = 'GraziosoSalvareLogo.png'  # Download from course materials
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    # FIXED: Logo + SNHU URL + Your Name (Dashboard Spec Requirement)
    html.Center([
        html.A(
            html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}),
            href='https://www.snhu.edu'  # REQUIRED: SNHU homepage link
        ),
        html.B(html.H1('CS-340 Dashboard - Kimberley Blackwood')),  # REQUIRED: Unique identifier
    ]),
    html.Hr(),
    
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([  
        html.Label("Select Rescue Type:", style={'fontSize': 20, 'margin': '10px'}),
        dcc.RadioItems(
            id='filter-type',
            options=[{'label': 'Reset', 'value': 'Reset'}] + 
                    [{'label': k, 'value': k} for k in RESCUE_BREEDS.keys()],
            value='Reset',
            labelStyle={'display': 'block', 'margin': '10px'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#f0f8ff'}),
    
    html.Hr(),
    
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
                         filter_action="native",           # Search box
                         sort_action="native",             # Sort columns
                         page_action="native",             # Pagination
                         page_size=15,                     # 15 rows/page
                         row_selectable="single",          # Row selection
                         column_selectable="single",       # Column highlight
                         selected_rows=[0],                # Auto-select first
                         style_cell={'textAlign': 'left', 'fontSize': 12, 'minWidth': '120px', 'padding': '5px'},
                         style_data_conditional=[
                             {'if': {'row_index': 'odd'}, 'backgroundColor': '#F8F9FA'}
                         ]
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
html.Div(className='row',
         style={'display': 'flex', 'width': '100%', 'height': '500px'},
         children=[
    html.Div(
        id='graph-id',
        style={'flex': '1', 'padding': '10px', 'height': '500px'},  # 50% width + fixed height
    ),
    html.Div(
        id='map-id',
        style={'flex': '1', 'padding': '10px', 'height': '500px'},  # 50% width + fixed height
    )
 ])
])
#############################################
# Interaction Between Components / Controller
#############################################

#FIX ME Add code to filter interactive data table with MongoDB queries
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIXED ME COMPLETE: MongoDB queries using EXACT spec criteria
    if filter_type == 'Reset':
        filtered_data = df.to_dict('records')
    else:
        config = RESCUE_BREEDS[filter_type]
        # COMPLETE QUERY: Breeds + Sex + Age (Dashboard Spec Table)
        query = {
            'breed': {'$in': config['breeds']},
            'sex_upon_outcome': config['sex'],  
            'age_upon_outcome_in_weeks': {
                '$gte': config['age_min'],     
                '$lte': config['age_max']
            }
        }
        filtered_df = pd.DataFrame.from_records(db.read(query))
        filtered_df.drop(columns=['_id'], inplace=True, errors='ignore')
        filtered_data = filtered_df.to_dict('records')
    
    return filtered_data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # Dynamic pie chart
    if viewData is None or len(viewData) == 0:
        return html.P("No data to display")
    
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Rescue Dog Breeds Distribution')
    
    return [
        dcc.Graph(figure=fig)
    ]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'backgroundColor': '#D2F3FF',
        'fontWeight': 'bold'
    } for i in selected_columns or []]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return html.Div("Loading map...")
    elif index is None:
        # Default Austin map
        return [dl.Map(style={'width': '100%', 'height': '500px'}, 
                      center=[30.75,-97.48], zoom=10, children=[dl.TileLayer()])]
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Safe extraction with defaults (handles missing columns)
    try:
        lat = float(dff.iloc[row]['location_lat']) if pd.notna(dff.iloc[row]['location_lat']) else 30.75
        lon = float(dff.iloc[row]['location_long']) if pd.notna(dff.iloc[row]['location_long']) else -97.48
        breed = str(dff.iloc[row]['breed']) if pd.notna(dff.iloc[row]['breed']) else "Unknown"
        name = str(dff.iloc[row]['name']) if pd.notna(dff.iloc[row]['name']) else "Unknown"
    except:
        lat, lon, breed, name = 30.75, -97.48, "Unknown", "Unknown"
    
    # Austin TX is at [30.75,-97.48]
    return [
        html.Div(  # ← THIS FIXES IT!
            dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat,lon], zoom=13, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat,lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([
                        html.H1("Rescue Candidate"),
                        html.P(f"Name: {name}"),
                        html.P(f"Breed: {breed}")
                    ])
                ])
            ])
        )
    ]

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server(mode="inline", port=8052, debug=True)  # FIXED: inline mode for Jupyter!


---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
IndexError: list index out of range

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
IndexError: list index out of range

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
IndexError: list index out of range

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
IndexError: list index out of range

